# Exercise sheet B - Template

In [36]:
# Some imports
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla

## Exercise 1

Define the correct matrix $\mathbf{A}$ and $\mathbf{b}$ for use with, e.g., `np.linalg.lstsq` or `scipy.linalg.lstsq` (see documentation), then call the least-squares solver appropriately. 

**Note**: Both examples from the 
exercise sheet work the same way, just the matrices are different.

In [9]:
# UNCOMMENT AND COMPLETE
# A = np.array(...).astype('float32')
# b = np.array(...).astype('float32')
# ...

## Exercise 2

Implement the Karczmarz algorithm from the lecture (see [here](https://github.com/rkwitt/teaching/blob/master/SS21/IBCC/VO/AlgebraicReconstruction.pdf)) and solve both problems from Exercise 1 with this algorithm.

Below is some **helper code**, e.g., to normalize the system of linear equations. Say you have
$$
\mathbf{A} = 
\left(
\begin{matrix}
1 & 0 & 1 \\
0 & 0 & 1 \\
\end{matrix}
\right)
$$
and 
$$
\mathbf{b} = [3,4]^\top
$$
then `normalize_system` would work as follows:

In [18]:
A = np.array([[1,0,1],[0,0,1]]).astype('float32')
b = np.array([3,4]).astype('float32')
ret = normalize_system(A,b)
A_norm = ret[0]
b_norm = ret[1]
print(A_norm)
print(b_norm)

[[0.70710677 0.         0.70710677]
 [0.         0.         1.        ]]
[2.1213205 4.       ]


Use `normalize_system` to create a normalized $\mathbf{A}$ and $\mathbf{b}$ that you use when calling your implementation of the Kaczmarz algorithm.

### Termination criterion

As termination criterion compute

$$\mathbf{r} = \mathbf{b} - \mathbf{A}\mathbf{x}^{(k)}$$

and check

$$\|\mathbf{r}\|_2 \leq 10^{-6}$$

where the norm can be computed using `np.linalg.norm`.

**Note**: Inner products with `numpy` can easily be done either with `np.dot` or using the syntax `a @ b`.

### Initialization

Initialize $\mathbf{x}^{(0)}$ as the vector of **all zeros**.

In [1]:
def compute_row_norms(A):
    if sp.issparse(A):
        return spla.norm(A, axis=1)
    return np.linalg.norm(A, axis=1)


def normalize_matrix(A, row_norms):
    normalization_matrix = sp.diags(1 / row_norms)
    return normalization_matrix @ A


def normalize_system(A, b):
    if not sp.issparse(A):
        A = np.array(A)

    row_norms = compute_row_norms(A)
    A = normalize_matrix(A, row_norms=row_norms)
    b = np.array(b).ravel() / row_norms

    return A, b, row_norms

In [25]:
def kaczmarz(A,b):
    # YOUR CODE GOES HERE
    # return x_k
    pass

## Solution

### Exercise 1

In [30]:
A0 = np.array([
    [1,1,0,0],
    [0,0,1,1],
    [1,0,1,0],
    [0,1,0,1],
    [1,0,0,1]
]).astype('float32')
b0 = np.array([3,7,4,6,5]).astype('float32')

sol0 = np.linalg.lstsq(A0,b0, rcond=None)
print(sol0[0])


A1 = np.array([
    [1,1,0,0],
    [0,0,1,1],
    [0,1,0,0],
    [1,0,0,1],
    [0,0,1,0],
    [0,1,0,1],
    [1,0,1,0],
    [0,0,0,1],
    [0,1,1,0],
    [1,0,0,0]
]).astype('float32')
b1 = np.array([3,7,2,5,3,6,4,4,5,1]).astype('float32')
sol1 = np.linalg.lstsq(A1,b1, rcond=None)
print(sol1[0])

[1. 2. 3. 4.]
[1. 2. 3. 4.]


### Exercise 2

In [33]:
class Kaczmarz:
    def __init__(self, A, b, max_iter=100, tol=1e-12):

        # normalize system
        self._A, self._b, self._row_norms = normalize_system(A, b)        
        self._n_rows = self._A.shape[0]
        self._n_cols = self._A.shape[1]
        self._max_iter = max_iter
        self._tol = tol

    def _check_norm(self, xk):
        residual = self._b - self._A @ xk
        if np.linalg.norm(residual) < self._tol:
            return True
        return False
            
    def solve(self):
        
        xk = np.zeros(self._n_cols)
        
        n_iter = 1
        while not self._check_norm(xk):
            for k in range(self._n_rows):        
                # cyclic index select
                ik = k % self._n_rows
                
                ai = self._A[ik]
                bi = self._b[ik]
                xk = xk + (bi - ai @ xk) * ai

            n_iter += 1
            if n_iter > self._max_iter:
                print('Not converged in {} iterations'.format(self._max_iter))
                return xk
                
        return xk

In [35]:
solver = Kaczmarz(A0,b0,max_iter=5000,tol=1e-6)
x0 = solver.solve()
print(x0)

solver = Kaczmarz(A1,b1,max_iter=5000,tol=1e-6)
x1 = solver.solve()
print(x1)

[1.00000001 2.0000003  3.         4.00000007]
[1.         2.00000038 2.9999997  4.        ]
